In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from package.databases.initialize import initialize_memories
initialize_memories()

In [23]:
from package.databases.models import LongTerm
from package.databases.session import get_session, Depends
from sqlmodel import Session, select, text

In [24]:
from package.databases.management.manage import LongTermManagement

In [25]:
longterm1 = LongTerm(
    raw="lt1",
    meta={"source":"a"}
)

longterm2 = LongTerm(
    raw="lt2",
    meta={"source":"a"}
)

longterm3 = LongTerm(
    raw="lt3",
    meta={"source":"b"}
)

longterms = [
    longterm1,
    longterm2,
    longterm3
]

In [ ]:
ltm = LongTermManagement()

ltm.create_raws(longterms=longterms, session=Depends(get_session))

In [27]:
ltm.embed_texts(embed_method="raw", session=Depends(get_session))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [28]:
# vector = ltm.embedding.run(sentences=["lt1"])[0]
ltm.read_similar_text(query="lt1", embed_method="raw", session=Depends(get_session))

[LongTerm(enrich_embedding=None, raw_embedding=array([-0.02778562,  0.06150023, -0.02956138, ..., -0.03680928,
        -0.00058394, -0.0143111 ], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='bdfdc27d-40b6-45a0-842b-ce778ea38367', raw='lt1', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 52, 50, 340272), updated_at=datetime.datetime(2025, 6, 23, 16, 53, 21, 27822)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.03846592,  0.03602301, -0.01591731, ..., -0.03680482,
         0.015809  , -0.00518122], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='b8cd49da-e868-4a1e-a0b2-25000b39de73', raw='lt2', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 52, 50, 340272), updated_at=datetime.datetime(2025, 6, 23, 16, 53, 21, 27822)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.01583992,  0.0286282 , -0.02951318, ..., -0.0033121 ,
         0.00714771, -0.00320035], sh

In [15]:
def read_longterms(session:Session=Depends(get_session)):
    statement = select(LongTerm)
    results = session.exec(statement).all()
    session.close()
    return results

read_longterms()

[LongTerm(enrich_embedding=None, raw_embedding=array([-0.01583992,  0.0286282 , -0.02951318, ..., -0.0033121 ,
         0.00714771, -0.00320035], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='9329fbb7-40b5-4abe-978e-1d51787cb610', raw='lt3', meta={'source': 'b'}, created_at=datetime.datetime(2025, 6, 23, 16, 32, 33, 774115), updated_at=datetime.datetime(2025, 6, 23, 16, 32, 57, 148317)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.02778562,  0.06150023, -0.02956138, ..., -0.03680928,
        -0.00058394, -0.0143111 ], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='bae34b3f-7f52-4d7e-a480-2bff39b9f831', raw='lt1', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 32, 33, 774115), updated_at=datetime.datetime(2025, 6, 23, 16, 32, 57, 148317)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.03846592,  0.03602301, -0.01591731, ..., -0.03680482,
         0.015809  , -0.00518122], 

In [16]:
read_longterms()

[LongTerm(enrich_embedding=None, raw_embedding=array([-0.01583992,  0.0286282 , -0.02951318, ..., -0.0033121 ,
         0.00714771, -0.00320035], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='9329fbb7-40b5-4abe-978e-1d51787cb610', raw='lt3', meta={'source': 'b'}, created_at=datetime.datetime(2025, 6, 23, 16, 32, 33, 774115), updated_at=datetime.datetime(2025, 6, 23, 16, 32, 57, 148317)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.02778562,  0.06150023, -0.02956138, ..., -0.03680928,
        -0.00058394, -0.0143111 ], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='bae34b3f-7f52-4d7e-a480-2bff39b9f831', raw='lt1', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 32, 33, 774115), updated_at=datetime.datetime(2025, 6, 23, 16, 32, 57, 148317)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.03846592,  0.03602301, -0.01591731, ..., -0.03680482,
         0.015809  , -0.00518122], 

In [9]:
def read_similar_longterms(vector:list[float], sources:list[str]|None=None, limit:int=5, session:Session=Depends(get_session)):
    statement = select(LongTerm)
    if sources:
        statement.where(text("meta ->> 'source' = ANY(:sources)"))
    statement = statement.order_by(LongTerm.raw_embedding.l2_distance(vector)).limit(limit).params(sources=sources)
    results = session.exec(statement).all()
    session.close()
    return results

read_similar_longterms(vector=em.run(sentences=["lt1"])[0])

[LongTerm(enrich_embedding=None, raw_embedding=array([-0.02778562,  0.06150023, -0.02956138, ..., -0.03680928,
        -0.00058394, -0.0143111 ], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='cb8c291a-e234-4b87-bbf9-5aaecd63b04b', raw='lt1', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 19, 55, 815259), updated_at=datetime.datetime(2025, 6, 23, 16, 20, 1, 49932)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.03846592,  0.03602301, -0.01591731, ..., -0.03680482,
         0.015809  , -0.00518122], shape=(1024,), dtype=float32), combo=None, combo_embedding=None, enrich=None, id='e2f5f26b-b5b2-42bd-af6b-d984fd276d8c', raw='lt2', meta={'source': 'a'}, created_at=datetime.datetime(2025, 6, 23, 16, 19, 55, 815259), updated_at=datetime.datetime(2025, 6, 23, 16, 20, 1, 49932)),
 LongTerm(enrich_embedding=None, raw_embedding=array([-0.01583992,  0.0286282 , -0.02951318, ..., -0.0033121 ,
         0.00714771, -0.00320035], shap

In [29]:
from package.databases.destroy import drop_all_tables

drop_all_tables()

✅ All tables dropped.
